In [53]:
import pickle
import polars as pl
import pandas as pd
from glob import glob

In [54]:
kdprov='32'

In [55]:
data_imputation=glob(f'/data/ksa/04_Data_Preprocessing/{kdprov}/01_imputation/*_imputed_data.pkl')

In [56]:
ls_dt=[]
for j in data_imputation:
    with open(j,'rb') as file:
        y=pickle.load(file)
        y['idsubsegment']=y['idpoint'].apply(lambda y1: y1.split('#')[0])
        y=pl.from_pandas(y)
        ls_dt.append(y)

In [57]:
data_full=pl.concat(ls_dt)

In [60]:
dt_var=data_full.group_by(['periode','idsubsegment']).agg([
    pl.col("Sigma0_VH_db_imputation").var().alias("Sigma0VH_variance"),
    pl.col("Sigma0_VV_db_imputation").var().alias("Sigma0VV_variance")
])

In [86]:
dt_var_idsubsegment_period=dt_var.with_columns(
    ((pl.col("Sigma0VH_variance")/pl.col("Sigma0VV_variance"))/2).alias("mean_var"),
).to_pandas()

In [87]:
bridging_date=pd.read_excel("/data/ksa/03_Sampling/bridging.xlsx", dtype='object', sheet_name="periode_to_date")
bridging_ksa=pd.read_excel("/data/ksa/03_Sampling/bridging.xlsx", dtype='object')

In [88]:
dt_var_idsubsegment_period['periode_start']=dt_var_idsubsegment_period.periode.apply(lambda y: y.split('_')[0][4:])

In [89]:
dt_var_idsubsegment_period=dt_var_idsubsegment_period.sort_values(['idsubsegment','periode']).merge(bridging_ksa.query('is_kabisat==0')[['periode_start','obs_in_a_year']])

In [122]:
dt_var_idsubsegment_period['tahun']=dt_var_idsubsegment_period.periode.apply(lambda y:str(y.split('_')[0][2:4]))

In [123]:
dt_var_idsubsegment_period['bulan']=dt_var_idsubsegment_period['obs_in_a_year'].astype(str)

In [106]:
df_label = pd.read_csv("/data/raw/processed/relabelled_data_ksa.csv",dtype='object')
df_label.head(2)
df_label.describe()

,idsegmen,idsubsegmen,tahun,bulan,obs,nth,id_x,class
count,4407480,4407480,4407480,4407480,4407480,4407480,4407480,4394176
unique,20405,9,2,12,10,24,183645,9
top,947101006,A1,22,1,7.0,0,947101006C3,NP
freq,216,489720,2203740,367290,1085448,183645,24,1085448


In [113]:
df_label['idsubsegment']=df_label['id_x']

In [125]:
df_gab_var_class=dt_var_idsubsegment_period[['periode','idsubsegment','mean_var','tahun','bulan']].merge(
    df_label[['idsubsegment','tahun','bulan','obs']])

In [129]:
df_gab_var_class_test_train=df_gab_var_class.query('(tahun!="23") or (bulan not in ["11","10","12"])')

In [152]:
df_quantile=df_gab_var_class_test_train.groupby('obs')['mean_var'].quantile([.05,.1,.2,.3]).unstack().reset_index()
df_quantile.columns=['obs','p5','p10','p20','p30']

In [153]:
df_full=df_gab_var_class_test_train.merge(df_quantile)
df_full['less_q5']=df_full['mean_var']<df_full['p5']
df_full['less_q10']=df_full['mean_var']<df_full['p10']
df_full['less_q20']=df_full['mean_var']<df_full['p20']
df_full['less_q30']=df_full['mean_var']<df_full['p30']

In [154]:
df_full.to_pickle(f'/data/ksa/04_Data_Preprocessing/{kdprov}/02_variance_filtering/variance_filtering.pkl')

In [155]:
pd.pivot_table(df_full,index='obs',columns=['less_q5'],values='idsubsegment',aggfunc='count').reset_index().rename(columns={False:'less_q5_F',
                                                                                                                         True:'less_q5_T'})

less_q5,obs,less_q5_F,less_q5_T
0,0.0,12,1
1,1.0,63972,3367
2,12.0,875,47
3,2.0,49484,2605
4,3.0,70160,3693
5,4.0,88693,4669
6,5.0,35449,1866
7,6.0,1464,77
8,7.0,52503,2764
9,8.0,48014,2528


In [156]:
pd.pivot_table(df_full,index='obs',columns=['less_q10'],values='idsubsegment',aggfunc='count').reset_index().rename(columns={False:'less_q10_F',
                                                                                                                         True:'less_q10_T'})

less_q10,obs,less_q10_F,less_q10_T
0,0.0,11,2
1,1.0,60605,6734
2,12.0,829,93
3,2.0,46880,5209
4,3.0,66467,7386
5,4.0,84025,9337
6,5.0,33583,3732
7,6.0,1387,154
8,7.0,49740,5527
9,8.0,45487,5055


In [157]:
pd.pivot_table(df_full,index='obs',columns=['less_q20'],values='idsubsegment',aggfunc='count').reset_index().rename(columns={False:'less_q20_F',
                                                                                                                         True:'less_q20_T'})

less_q20,obs,less_q20_F,less_q20_T
0,0.0,10,3
1,1.0,53871,13468
2,12.0,737,185
3,2.0,41671,10418
4,3.0,59082,14771
5,4.0,74689,18673
6,5.0,29852,7463
7,6.0,1233,308
8,7.0,44213,11054
9,8.0,40433,10109


In [158]:
pd.pivot_table(df_full,index='obs',columns=['less_q30'],values='idsubsegment',aggfunc='count').reset_index().rename(columns={False:'less_q30_F',
                                                                                                                         True:'less_q30_T'})

less_q30,obs,less_q30_F,less_q30_T
0,0.0,9,4
1,1.0,47137,20202
2,12.0,645,277
3,2.0,36462,15627
4,3.0,51697,22156
5,4.0,65353,28009
6,5.0,26120,11195
7,6.0,1079,462
8,7.0,38687,16580
9,8.0,35379,15163
